---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load excel dataset
energy = pd.read_excel('Energy Indicators.xls', header=0, skip_footer=1)

#Drop invalid data indices
energy.drop(energy.index[list(range(0,16))+list(range(243, 280))], axis=0, inplace=True)

#Since removing these indices result in a dataframe not starting from 0
#Therefore, reset the index
energy.reset_index(inplace=True)

#Drop the unrequired columns and then rename existing columns
energy.drop(['index', 'Unnamed: 0', 'Unnamed: 1'], axis=1, inplace=True)
energy.rename(columns={'Environmental Indicators: Energy':'Country', 'Unnamed: 3':'Energy Supply', 
                           'Unnamed: 4':'Energy Supply per Capita', 'Unnamed: 5':'% Renewable'}, inplace=True)

#Replace '...' with NaN
energy.replace("...", np.nan, inplace=True)

#Replace the strings
energy.replace({"Republic of Korea": "South Korea", "United States of America": "United States",
            "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", 
            "China, Hong Kong Special Administrative Region": "Hong Kong"}, regex=True, inplace=True)

#Replace digits from the country names
energy['Country'].replace('\d', '', regex=True, inplace=True)

#Replace the parenthesis area from country names
#\s is used to remove space before parenthesis, like in example of 'Iran'
energy['Country'].replace('\s*\([^)]+\)', '', regex=True, inplace=True)


#Converting unit of energy supply
energy['Energy Supply'] *= 1000000

#Load GDP dataset
GDP = pd.read_csv('world_bank.csv', header=None)

#Remove rows and columns
GDP.drop(list(range(0,5)), axis=0, inplace=True)
GDP.drop(list(range(2,50)), axis=1, inplace=True)

#Replace words
GDP.replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}, 
                inplace=True)

#Reset index
GDP.reset_index(inplace=True)
GDP.drop('index', axis=1, inplace=True)
GDP.rename(columns={0:"Country", 1:"Country Code", 50:"2006", 51:"2007", 52:"2008", 53:"2009", 54:"2010",
                   55:"2011", 56:"2012", 57:"2013", 58:"2014", 59:"2015"}, inplace=True)

#Load country rank dataset
ScimEn = pd.read_excel('scimagojr-3.xlsx')   

In [3]:
def answer_one():        
    
    #merge the 3 datasets
    df = pd.merge(ScimEn, pd.merge(GDP, energy, how='outer', on='Country'), how='outer', on='Country')
    df.drop('Country Code', axis=1, inplace=True)
    df.set_index('Country', inplace=True)
    df.sort('Rank', inplace=True)
    df = df[:15]
    df[['Rank', 'H index']] = df[['Rank', 'H index']].astype('int64')
    
    return df

In [4]:
answer_one()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Energy Supply,Energy Supply per Capita,% Renewable
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050.0,126767.0,597237.0,411683.0,4.70,138,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,1.271910e+11,93.0,19.754910
United States,2,96661.0,94747.0,792274.0,265436.0,8.20,230,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,9.083800e+10,286.0,11.570980
Japan,3,30504.0,30287.0,223024.0,61554.0,7.31,134,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,1.898400e+10,149.0,10.232820
United Kingdom,4,20944.0,20357.0,206091.0,37874.0,9.84,139,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,7.920000e+09,124.0,10.600470
Russian Federation,5,18534.0,18301.0,34266.0,12422.0,1.85,57,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,3.070900e+10,214.0,17.288680
Canada,6,17899.0,17620.0,215003.0,40930.0,12.01,149,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,1.043100e+10,296.0,61.945430
Germany,7,17027.0,16831.0,140566.0,27426.0,8.26,126,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,1.326100e+10,165.0,17.901530
India,8,15005.0,14841.0,128763.0,37209.0,8.58,115,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,3.319500e+10,26.0,14.969080
France,9,13153.0,12973.0,130632.0,28601.0,9.93,114,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,1.059700e+10,166.0,17.020280


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [5]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [6]:
def answer_two():
    
    union = pd.merge(ScimEn, pd.merge(GDP, energy, how='outer', on='Country'), how='outer', on='Country')
    intersect = pd.merge(ScimEn, pd.merge(GDP, energy, how='inner', on='Country'), how='inner', on='Country')
    
    #print(len(intersect))
    #print(len(union))
    
    return len(union)-len(intersect)

In [7]:
answer_two()

156

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [8]:
def answer_three():
    Top15 = answer_one()
    avgGDP = pd.Series(Top15[['2006','2007','2008','2009','2010','2011','2012','2013',
                              '2014','2015']].mean(axis=1, skipna=True), 
                       name='avgGDP')
    avgGDP.sort_values(inplace=True, ascending=False)
    return avgGDP

In [9]:
answer_three()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [10]:
def answer_four():
    Top15 = answer_one()
    avgGDP_6th = answer_three().reset_index()
    country = avgGDP_6th.iloc[5]['Country']
    changedGDP = Top15.loc[country]['2015'] - Top15.loc[country]['2006']
    return changedGDP

In [11]:
answer_four()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [12]:
def answer_five():
    Top15 = answer_one()
    meanEnergyPerCapita = Top15['Energy Supply per Capita'].mean(axis=0)
    return meanEnergyPerCapita

In [13]:
answer_five()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [14]:
def answer_six():
    Top15 = answer_one()
    perRenewable = pd.Series(Top15['% Renewable'], name='perRenewable')
    perRenewable.sort_values(inplace=True, ascending=False)
    df_Renewable = perRenewable.reset_index()
    country_name = df_Renewable.iloc[0]['Country']
    country_perRenewable = df_Renewable.iloc[0]['perRenewable']
    return (country_name, country_perRenewable)

In [15]:
answer_six()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [16]:
def answer_seven():
    Top15 = answer_one()
    Top15['Ratio_selfcite_totalcite'] = Top15['Self-citations'] / Top15['Citations']
    ratioTally = pd.Series(Top15['Ratio_selfcite_totalcite'], name='ratioTally')
    ratioTally.sort_values(inplace=True, ascending=False)
    df_ratio = ratioTally.reset_index()
    country_name = df_ratio.iloc[0]['Country']
    country_ratio = df_ratio.iloc[0]['ratioTally']
    return (country_name, country_ratio)

In [17]:
answer_seven()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [18]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population(Estd.)'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    population = pd.Series(Top15['Population(Estd.)'], name='Population')
    population.sort_values(inplace=True, ascending=False)    
    df_pop = population.reset_index()
    country_name = df_pop.iloc[2]['Country']
    return country_name

In [19]:
answer_eight()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [39]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    return Top15['Energy Supply per Capita'].corr(Top15['Citable docs per Capita'])

In [40]:
answer_nine()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


0.79400104354429424

In [22]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [23]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [47]:
def answer_ten():
    Top15 = answer_one()
    median_top15 = Top15['% Renewable'].median(axis=0)
    Top15['HighRenew'] = [1 if x >= median_top15 else 0 for x in Top15['% Renewable']]
    HighRenew = pd.Series(Top15['HighRenew'], name='HighRenew')
    return HighRenew

In [48]:
answer_ten()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [26]:
def answer_eleven():
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15 = answer_one()
    
    df_continent = pd.Series(ContinentDict, name='Continent').reset_index()
    df_continent.rename(columns={'index':'Country'}, inplace=True)
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    df_popest_dict = pd.merge(Top15[['PopEst']], df_continent, how='outer', right_on='Country', left_index=True)
    df_agg = df_popest_dict.dropna().groupby('Continent').agg({'PopEst':[np.size,np.sum, np.mean, np.std]})
    df_agg.columns = df_agg.columns.droplevel()
    
    return df_agg

In [27]:
answer_eleven()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [28]:
def answer_twelve():
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15 = answer_one()
    Top15['Continent'] = Top15.index.to_series().map(ContinentDict)
    Top15['% Renewable'] = pd.cut(Top15['% Renewable'], 5)
    Top15_grouped = Top15.groupby(['Continent', '% Renewable']).size()
    return Top15_grouped

In [29]:
answer_twelve()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Continent      % Renewable     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [49]:
def answer_thirteen():
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    #Top15['PopEst'] = Top15['PopEst'].astype('str')
    PopEst = pd.Series(['{:,}'.format(x) for x in Top15['PopEst']], name='PopEst', index=Top15.index)
    
    return PopEst

In [50]:
answer_thirteen()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [32]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [33]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!